In [1]:
# Imports
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision.transforms import Compose, Resize, ToTensor
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import os
import nltk
import pandas as pd
import numpy as np
import spacy
from collections import Counter
import matplotlib.pyplot as plt

from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

from PIL import Image
import matplotlib.pyplot as plt

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

Vocabulary Class

In [3]:
class Vocabulary:
    def __init__(self, freq_threshold):
        self.freq_threshold = freq_threshold
        self.itos = {0: "<PAD>", 1: "<SOS>", 2: "<EOS>", 3: "<UNK>"}
        self.stoi = {v: k for k, v in self.itos.items()}
    
    def __len__(self):
        return len(self.itos)
    
    @staticmethod
    def tokenizer_eng(text):
        return [tok.text.lower() for tok in spacy.load("en_core_web_sm").tokenizer(text)]
    
    def build_vocabulary(self, sentence_list):
        frequencies = Counter()
        idx = 4

        for sentence in sentence_list:
            for word in self.tokenizer_eng(sentence):
                frequencies[word] += 1
                if frequencies[word] == self.freq_threshold:
                    self.stoi[word] = idx
                    self.itos[idx] = word
                    idx += 1
    
    def numericalize(self, text):
        tokenized_text = self.tokenizer_eng(text)
        return [
            self.stoi.get(token, self.stoi["<UNK>"])
            for token in tokenized_text
        ]

We will be using the Flickr8k dataset.

In [4]:
class FlickrDataset(Dataset):
    def __init__(self, root_dir, captions_file, transform=None, freq_threshold=5):
        self.df = pd.read_csv(captions_file)
        self.root_dir = root_dir
        self.transform = transform

        self.imgs = self.df["image"].tolist()
        raw_captions = self.df["caption"].tolist()
        
        self.vocab = Vocabulary(freq_threshold)
        self.vocab.build_vocabulary(raw_captions)

        # Preprocess all captions once
        self.captions = []
        for cap in raw_captions:
            tokens = [self.vocab.stoi["<SOS>"]]
            tokens += self.vocab.numericalize(cap)
            tokens.append(self.vocab.stoi["<EOS>"])
            self.captions.append(torch.tensor(tokens))

    def __len__(self):
        return len(self.captions)
    
    def __getitem__(self, idx):
        img_path = os.path.join(self.root_dir, self.imgs[idx])
        img = Image.open(img_path).convert("RGB")

        if self.transform:
            img = self.transform(img)

        caption = self.captions[idx]
        return img, caption

In [5]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

def collate_fn(batch):
    images = []
    captions = []
    for img, cap in batch:
        images.append(img)
        captions.append(cap)
    
    images = torch.stack(images)
    captions = nn.utils.rnn.pad_sequence(captions, batch_first=True, padding_value=0)
    return images, captions

Modelle: ResNet50 for Image Feature Extraction and LSTM for language processing

In [6]:
class EncoderCNN(nn.Module):
    def __init__(self, embed_size):
        super(EncoderCNN, self).__init__()
        resnet = models.resnet50(pretrained=True)
        for param in resnet.parameters():
            param.requires_grad = False
        self.resnet = nn.Sequential(*list(resnet.children())[:-1])
        self.fc = nn.Linear(resnet.fc.in_features, embed_size)
    
    def forward(self, images):
        with torch.no_grad():
            features = self.resnet(images)
        features = features.reshape(features.size(0), -1)
        return self.fc(features)

class DecoderRNN(nn.Module):
    def __init__(self, embed_size, hidden_size, vocab_size, num_layers=1):
        super(DecoderRNN, self).__init__()
        self.embed = nn.Embedding(vocab_size, embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, features, captions):
        embeddings = self.embed(captions[:, :-1])
        inputs = torch.cat((features.unsqueeze(1), embeddings), dim=1)
        hiddens, _ = self.lstm(inputs)
        outputs = self.fc(hiddens)
        return outputs

Helper functions

In [7]:
def train(model, dataloader, criterion, optimizer, vocab_size, num_epochs=5):
    for epoch in range(num_epochs):
        print(f"Start epoch {epoch}")
        for idx, (imgs, captions) in enumerate(dataloader):
            imgs, captions = imgs.to(device), captions.to(device)

            features = encoder(imgs)
            outputs = decoder(features, captions)

            loss = criterion(outputs.view(-1, vocab_size), captions[:, 1:].reshape(-1))

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

In [8]:
def caption_image(encoder, decoder, image, vocab, max_length=50):
    result_caption = []

    with torch.no_grad():
        x = encoder(image.unsqueeze(0).to(device))
        states = None
        word = torch.tensor([vocab.stoi["<SOS>"]]).to(device)

        for _ in range(max_length):
            embedding = decoder.embed(word).unsqueeze(1)
            hiddens, states = decoder.lstm(embedding, states)
            output = decoder.fc(hiddens.squeeze(1))
            predicted = output.argmax(1)

            word = predicted
            predicted_word = vocab.itos[predicted.item()]
            if predicted_word == "<EOS>":
                break
            result_caption.append(predicted_word)

    return " ".join(result_caption)

Download Flickr8k dataset:
https://www.kaggle.com/datasets/adityajn105/flickr8k

Convert txt file to csv file. Goal is to use csv directly in the dataloader

In [9]:
# txt_path = "Flickr8k_text/captions.txt"
# csv_path = "Flickr8k_text/captions.csv"

In [10]:
# df = pd.read_csv(txt_path)
# df.to_csv(csv_path, index=False)

In [ ]:
# Load dataset and create dataloaders
dataset = FlickrDataset(
    root_dir="Flickr8k_Dataset",
    captions_file="Flickr8k_text/captions.csv",
    transform=transform
)
loader = DataLoader(dataset, batch_size=32, shuffle=True, collate_fn=collate_fn, num_workers=4)

In [ ]:
# Hyperparameter
embed_size = 256
hidden_size = 512
vocab_size = len(dataset.vocab)
num_epochs = 10
learning_rate = 3e-4

# Models
encoder = EncoderCNN(embed_size).to(device)
decoder = DecoderRNN(embed_size, hidden_size, vocab_size).to(device)

# Optimizer & Loss
criterion = nn.CrossEntropyLoss(ignore_index=dataset.vocab.stoi["<PAD>"])
params = list(decoder.parameters()) + list(encoder.fc.parameters())
optimizer = torch.optim.Adam(params, lr=learning_rate)

# Training
train((encoder, decoder), loader, criterion, optimizer, vocab_size, num_epochs)

Caption Generation

In [ ]:
def show_caption(image_path, encoder, decoder, vocab, transform):
    image = Image.open(image_path).convert("RGB")
    img_tensor = transform(image).to(device)

    caption = caption_image(encoder, decoder, img_tensor, vocab)
    
    plt.imshow(image)
    plt.title(caption)
    plt.axis("off")
    plt.show()

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Use picture from Flickr8k_Dataset
image_path = "Flickr8k_Dataset/2513260012_03d33305cf.jpg"
show_caption(image_path, encoder, decoder, dataset.vocab, transform)